In [1]:
import pyrebase
import uuid

from core.config import db

problems = db.child("user").get()

# Iterate through the retrieved data
for problem in problems.each():
    print(problem.key(), problem.val())

-OCtVr9MP27UrtIN44zw {'cart': ['book1', 'book2'], 'date': '2024-11-29', 'gender': True, 'history': ['order1', 'order2'], 'id': '1', 'info': 'System Administrator', 'managed_users': ['2', '3', '4'], 'name': 'John Doe', 'password': 'securepassword', 'permissions': ['manage_users', 'view_reports', 'edit_content'], 'phone': '0123456789', 'role': 'Admin', 'username': 'admin123'}
-OCtgV62_YOYSYcme8Qs {'date': '2024-11-29', 'gender': True, 'history': ['delivery1', 'delivery2'], 'id': '3', 'info': 'Delivery Specialist', 'name': 'Mike Johnson', 'password': 'deliversecure', 'phone': '0123456789', 'region': 'Downtown', 'role': 'DeliveryMan', 'username': 'delivery_mike'}


In [2]:
from models.users import  create_user

admin_data = {
    "id": "1",
    "name": "John Doe",
    "username": "admin123",
    "password": "securepassword",
    "date": "2024-11-29",
    "phone": "0123456789",
    "gender": True,
    "info": "System Administrator",
    "role": "Admin",
    "permissions": ["manage_users", "view_reports", "edit_content"],
    "managed_users": ["2", "3", "4"],
}

# Dữ liệu mẫu cho RegularUser
user_data = {
    "id": "2",
    "name": "Jane Smith",
    "username": "jane_smith",
    "password": "userpassword",
    "date": "2024-11-29",
    "phone": "0123456789",
    "gender": False,
    "info": "Regular customer",
    "role": "User",
    "cart": ["book1", "book2"],
    "history": ["order1", "order2"],
    # "subscription_type": "Premium",
    # "wishlist": ["book3", "book4"],
}

# Dữ liệu mẫu cho DeliveryMan
delivery_man_data = {
    "id": "3",
    "name": "Mike Johnson",
    "username": "delivery_mike",
    "password": "deliversecure",
    "date": "2024-11-29",
    "phone": "0123456789",
    "gender": True,
    "info": "Delivery Specialist",
    "role": "DeliveryMan",
    "history": ["delivery1", "delivery2"],
    # "vehicle": "Motorbike",
    "region": "Downtown",
    # "completed_deliveries": 120,
    # "rating":[5, 5]
}

# Tạo đối tượng bằng Factory Method
admin = create_user(admin_data)
user1 = create_user(user_data)
delivery_man = create_user(delivery_man_data)

# Kiểm tra kết quả
print(admin)
print(user1)
print(delivery_man)

id='1' name='John Doe' username='admin123' password='securepassword' date='2024-11-29' gender=True info='System Administrator' phone='0123456789' role='Admin' permissions=['manage_users', 'view_reports', 'edit_content'] managed_users=['2', '3', '4']
id='2' name='Jane Smith' username='jane_smith' password='userpassword' date='2024-11-29' gender=False info='Regular customer' phone='0123456789' role='User' cart=['book1', 'book2'] history=['order1', 'order2']
id='3' name='Mike Johnson' username='delivery_mike' password='deliversecure' date='2024-11-29' gender=True info='Delivery Specialist' phone='0123456789' role='DeliveryMan' history=['delivery1', 'delivery2'] region='Downtown'


In [ ]:
# # Chuyển đối tượng RegularUser thành dictionary
# user_dict = user1.dict()

# # Đẩy dữ liệu lên Firebase
# db.child("user").push(user_dict)


{'name': '-OCtHDbLiMdhCo830cW9'}

In [ ]:
from repositories.db_repository import FirebaseService
from models.users import User, create_user
from collections import OrderedDict


class UserRepository:
    def __init__(self):
        self.collection_name = "user"

    def add_user(self, user_data: dict):
        # Tạo đối tượng User dựa trên role
        user = create_user(user_data)
        key = user.id
        FirebaseService.add_data(self.collection_name, key, user.dict())
        return {"message": "User created successfully."}

    def get_all_users(self):
        # Lấy tất cả người dùng từ Firebase
        users_data = FirebaseService.get_data(self.collection_name)
        # return [create_user(user) for user in users_data]
        if isinstance(users_data, OrderedDict):
        # Duyệt qua các giá trị trong OrderedDict và tạo đối tượng User
            return [create_user(user_data) for user_data in users_data.values()]
        else:
            raise ValueError("Data from Firebase is not in expected format")


    def get_user_by_id(self, user_id: str):
        users_data = FirebaseService.get_data_by_key(self.collection_name, user_id)
        # user_get = db.child(self.collection_name).order_by_child("id").equal_to(user_id).get()
        # # print(user_get)
        # for user in user_get.each():
        #     # print(user.key())  # In khóa của mỗi bản ghi
        #     # print(user.val()) 
        #     return create_user(user.val())
        return create_user(users_data.val())

    def update_user(self, user_id: str, user_data: dict):
        # Tạo đối tượng User và cập nhật dữ liệu
        usernew = create_user(user_data)
        # user_get = db.child(self.collection_name).order_by_child("id").equal_to(user_id).get()
        # # print(user_get)
        # for user in user_get.each():
        #     tmp_key = user.key()
        #     db.child("user").child(tmp_key).update(usernew.dict())
        FirebaseService.update_data(self.collection_name, user_id, usernew.dict())
        return {"message": "User updated successfully."}

    def delete_user(self, user_id: str):
        FirebaseService.delete_data(self.collection_name, user_id)
        return {"message": "User deleted successfully."}


# Khởi tạo một instance cho repository
user_repository = UserRepository()


In [4]:
user = create_user(user_data)
key = user.id
FirebaseService.add_data("user", key, user.dict())

In [5]:
user_repository.add_user(user_data)

{'message': 'User created successfully.'}

In [4]:
all_users = user_repository.get_all_users()
print(all_users)

[Admin(id='2', name='John Doe', username='admin123', password='securepassword', date='2024-11-29', gender=True, info='System Administrator', phone='0123456789', role='Admin', permissions=['manage_users', 'view_reports', 'edit_content'], managed_users=['2', '3', '4']), Admin(id='1', name='John Doe', username='admin123', password='securepassword', date='2024-11-29', gender=True, info='System Administrator', phone='0123456789', role='Admin', permissions=['manage_users', 'view_reports', 'edit_content'], managed_users=['2', '3', '4']), DeliveryMan(id='3', name='Mike Johnson', username='delivery_mike', password='deliversecure', date='2024-11-29', gender=True, info='Delivery Specialist', phone='0123456789', role='DeliveryMan', history=['delivery1', 'delivery2'], region='Downtown')]


In [8]:
all_users=user_repository.get_user_by_id( "1")
print(all_users)

id='1' name='John Doe' username='admin123' password='securepassword' date='2024-11-29' gender=True info='System Administrator' phone='0123456789' role='Admin' permissions=['manage_users', 'view_reports', 'edit_content'] managed_users=['2', '3', '4']


In [16]:
collection = "user"
key = "1"
user_get = db.child(collection).order_by_child("id").equal_to(key).get()
# print(user_get)
for user in user_get.each():
    print(user.key())  # In khóa của mỗi bản ghi
    print(user.val()) 
    create_user(user.val())

-OCtVr9MP27UrtIN44zw
{'date': '2024-11-29', 'gender': True, 'id': '1', 'info': 'System Administrator', 'managed_users': ['2', '3', '4'], 'name': 'John Doe', 'password': 'securepassword', 'permissions': ['manage_users', 'view_reports', 'edit_content'], 'phone': '0123456789', 'role': 'Admin', 'username': 'admin123'}


In [13]:
users_data1 = FirebaseService.get_data_by_key("user","2")
print(users_data1)

OrderedDict([('-OCtUuomHx8WrsFz8EnQ', {'date': '2024-11-29', 'gender': True, 'id': '2', 'info': 'System Administrator', 'managed_users': ['2', '3', '4'], 'name': 'John Doe', 'password': 'securepassword', 'permissions': ['manage_users', 'view_reports', 'edit_content'], 'phone': '0123456789', 'role': 'Admin', 'username': 'admin123'})])


In [13]:
user_repository.update_user("2",admin_data)

{'message': 'User updated successfully.'}

In [12]:
user_repository.update_user("1",user_data)

{'message': 'User updated successfully.'}

In [ ]:
FirebaseService.update_data("user", "1",user_data)

In [4]:
user_repository.delete_user("2")

{'message': 'User deleted successfully.'}

## Books

In [1]:
import pyrebase
import uuid

from core.config import db

In [35]:
book1 = {
        "id": 111111,
        "name": "Nhà Giả Kim",
        "price": 71100,
        "description": "Nhà Giả Kim Tất cả những trải nghiệm trong chuyến phiêu du theo đuổi vận mệnh của mình đã giúp Santiago thấu hiểu được ý nghĩa sâu xa nhất của hạnh phúc, hòa hợp với vũ trụ và con người.\nTiểu thuyết Nhà giả kim của Paulo Coelho như một câu chuyện cổ tích giản dị, nhân ái, giàu chất thơ, thấm đẫm những minh triết huyền bí của phương Đông. Trong lần xuất bản đầu tiên tại Brazil vào năm 1988, sách chỉ bán được 900 bản. Nhưng, với số phận đặc biệt của cuốn sách dành cho toàn nhân loại, vượt ra ngoài biên giới quốc gia, Nhà giả kim đã làm rung động hàng triệu tâm hồn, trở thành một trong những cuốn sách bán chạy nhất mọi thời đại, và có thể làm thay đổi cuộc đời người đọc.\n“Nhưng nhà luyện kim đan không quan tâm mấy đến những điều ấy. Ông đã từng thấy nhiều người đến rồi đi, trong khi ốc đảo và sa mạc vẫn là ốc đảo và sa mạc. Ông đã thấy vua chúa và kẻ ăn xin đi qua biển cát này, cái biển cát thường xuyên thay hình đổi dạng vì gió thổi nhưng vẫn mãi mãi là biển cát mà ông đã biết từ thuở nhỏ. Tuy vậy, tự đáy lòng mình, ông không thể không cảm thấy vui trước hạnh phúc của mỗi người lữ khách, sau bao ngày chỉ có cát vàng với trời xanh nay được thấy chà là xanh tươi hiện ra trước mắt. ‘Có thể Thượng đế tạo ra sa mạc chỉ để cho con người biết quý trọng cây chà là,’ ông nghĩ.”\n- Trích Nhà giả kimGiá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Bên cạnh đó, tuỳ vào loại sản phẩm, hình thức và địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển, phụ phí hàng cồng kềnh, thuế nhập khẩu (đối với đơn hàng giao từ nước ngoài có giá trị trên 1 triệu đồng).....",
        "authors": [
            {
                "id": 9173,
                "name": "Paulo Coelho"
            }
        ],
        "quantity": 37,
        "image_urls": [
            "https://salt.tikicdn.com/cache/w1200/ts/product/f4/f9/1f/9d737e0491083a2b9d9de2ca86fef84d.jpg",
            "https://salt.tikicdn.com/cache/w1200/ts/product/83/30/87/737846efdb9f28f0f51352cacf9225c5.jpg"
        ],
        "rating_average": 4.7,
        "review_count": 5,
        "quantity_sold": 7,
        "category": "Tiểu Thuyết"
    }

book2 = {
        "id": 11111212311,
        "name": "21 Nguyên Tắc Tự Do Tài Chính (Tái Bản)",
        "price": 46000,
        "description": "Không ai tốt bằng bạn và không ai thông minh hơn bạn. Những người thành công đơn giản là những người bình thường đã luyện tập và áp dụng thành công 21 nguyên tắc này để đạt được tự do tài chính. Và bạn hoàn toàn có thể giống họ. Triệu phú cũng là người bình thường, nhưng điểm khác biệt ở chỗ họ đã áp dụng 21 nguyên tắc này trong cuộc sống và công việc của mình.\n\"21 Nguyên tắc tự do tài chính\" được viết với sự tóm lược ra ngắn ngọn và dễ hiểu của ông Brian Trancy. Với các nguyên tắc từ việc thay đổi suy nghĩ, đến đi vào tính hành động và hành động ngay lập tức, rồi tư duy rộng để đưa bản thân chạm đến sự thành công lớn với chính mình. Vậy 21 nguyên tắc đó là gì: \n1. Mơ những giấc mơ lớn\n2. Đưa ra chỉ dẫn rõ ràng\n3. Coi mình là một người chủ\n4. Hãy làm những điều bạn yêu thích\n5. Cam kết với sự hoàn hảo\n6. Làm việc lâu hơn, chăm chỉ hơn\n7. Học tập suốt đời\n8. Trước hết hãy trả công cho mình\n9. Học mọi chi tiết về công việc của mình\n10. Tận tâm phụng sự cho người khác\n11. Thành thật tuyệt đối với chính mình và với người khác\n12. Xác định điều ưu tiên cao nhất và tập trung toàn bộ tâm trí vào đó\n13. Tạo dựng danh tiếng về tốc độ và độ tin cậy\n14. Chuẩn bị để leo từ đỉnh cao này tới đỉnh cao kia\n15. Tự rèn luyện kỷ luật trong tất cả mọi việc\n16. Khai mở khả năng sáng tạo thiên bẩm của bạn\n17. Giao du với đúng người\n18. Hãy chăm sóc sức khoẻ thể chất của bạn\n19. Quyết đoán và xu hướng hành động\n20. Không bao giờ cho phép thất bại trở thành lựa chọn\n21. Vượt qua \"thử thách tính kiên trì\"\nCuốn sách nhỏ này sẽ cung cấp cho bạn phương thức để bạn có thể trở thành triệu phú, cho dù bạn là ai và có xuất phát điểm hiện tại như thế nào. Bạn sẽ học được cách xác lập mục tiêu, lên kế hoạch và tổ chức hành động để có thể đạt được nhiều mục tiêu tài chính hơn những gì bạn mong muốn.Giá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Bên cạnh đó, tuỳ vào loại sản phẩm, hình thức và địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển, phụ phí hàng cồng kềnh, thuế nhập khẩu (đối với đơn hàng giao từ nước ngoài có giá trị trên 1 triệu đồng).....",
        "authors": [
            {
                "id": 3673,
                "name": "Brian Tracy"
            }
        ],
        "quantity": 70,
        "image_urls": [
            "https://salt.tikicdn.com/cache/w1200/ts/product/16/4b/7a/79be0c1ab963a8bc29961ada3d3a633d.jpg",
            "https://salt.tikicdn.com/cache/w1200/ts/product/97/85/08/b321d2cba71ebd3def481d4f780bcf49.jpg"
        ],
        "rating_average": 5,
        "review_count": 3,
        "quantity_sold": 3,
        "category": "Sách tài chính"
    }

book3 = {
        "id": 145289356,
        "name": "Sách Atomic Habits - Thay Đổi Tí Hon, Hiệu Quả Bất Ngờ ( Những Thay Đổi Nhỏ Tạo Nên Thành Công Lớn/ Tặng kèm Bookmark)",
        "price": 170000,
        "description": "Atomic Habits - Thay Đổi Tí Hon, Hiệu Quả Bất Ngờ\n•  Wall Street Journal Bestseller, USA Today Bestseller, Publisher's Weekly Bestseller•  Nằm trong Top 20 tựa sách thể loại non-fiction bán chạy và được tìm đọc nhiều nhất của Amazon suốt 40 tuần tính đến tháng 9/2019\nMột thay đổi tí hon có thể biến đổi cuộc đời bạn không?\nHẳn là khó đồng ý với điều đó. Nhưng nếu bạn thay đổi thêm một chút? Một chút nữa? Rồi thêm một chút nữa? Đến một lúc nào đó, bạn phải công nhận rằng cuộc sống của mình đã chuyển biến nhờ vào một thay đổi nhỏ\nVà đó chính là sức mạnh của thói quen nguyên tử.Giá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Bên cạnh đó, tuỳ vào loại sản phẩm, hình thức và địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển, phụ phí hàng cồng kềnh, thuế nhập khẩu (đối với đơn hàng giao từ nước ngoài có giá trị trên 1 triệu đồng).....",
        "authors": [],
        "quantity": 143,
        "image_urls": [
            "https://salt.tikicdn.com/cache/w1200/ts/product/9f/81/ed/27bf5e6d1e5cb3aa8425ee8fc62aa652.png",
            "https://salt.tikicdn.com/cache/w1200/ts/product/e2/32/b3/085ee546b06b792e847edd88b02b8f15.jpg"
        ],
        "rating_average": 5,
        "review_count": 4,
        "quantity_sold": 8,
        "category": "Sách tư duy - Kỹ năng sống"
    },

In [ ]:
from repositories.db_repository import FirebaseService
from models.book import Book, create_book
from collections import OrderedDict
from pydantic import ValidationError


class BookRepository:
    def __init__(self):
        self.collection_name = "books"

    def add_book(self, book_data: dict):
        # Tạo đối tượng book dựa trên role
        book = create_book(book_data)
        key = book.id
        FirebaseService.add_data(self.collection_name, key, book.dict())
        return {"message": "book created successfully."}


    def get_all_books(self):
        """
        Lấy tất cả sách từ Firebase và bỏ qua các sách thiếu name hoặc price.
        """
        # Lấy dữ liệu từ Firebase
        books_data = FirebaseService.get_data(self.collection_name)
        
        if isinstance(books_data, OrderedDict):
            # Duyệt qua từng sách, chỉ giữ sách hợp lệ
            valid_books = []
            for book_data in books_data.values():
                # Bỏ qua sách không có 'name' hoặc 'price'
                if not book_data.get('name') or not book_data.get('price'):
                    continue
                # Tạo đối tượng Book từ dữ liệu hợp lệ
                valid_books.append(create_book(book_data))
                # try:
                #     valid_books.append(create_book(book_data))
                # except ValidationError as e:
                #     print(f"Skipping invalid book data: {book_data}")
            
            return valid_books
        else:
            raise ValueError("Data from Firebase is not in expected format")

    def get_book_by_id(self, book_id: int):
        books_data = FirebaseService.get_data_by_key(self.collection_name, book_id)
        return create_book(books_data)

    def update_book_byID(self, book_id: int, book_data: dict):
        FirebaseService.update_data(self.collection_name, book_id, book_data)
        return {"message": "book updated successfully."}
    
    def update_book(self, book_data: dict):
        # Tạo đối tượng book và cập nhật dữ liệu
        book = create_book(book_data)
        key = book.id
        FirebaseService.update_data(self.collection_name, key, book.dict())
        return {"message": "book updated successfully."}

    def delete_book(self, book_id: int):
        FirebaseService.delete_data(self.collection_name, book_id)
        return {"message": "book deleted successfully."}


# Khởi tạo một instance cho repository
book_repository = BookRepository()


In [4]:
all_books = book_repository.get_all_books()
print(all_books[:15])

[Book(id=341073, name='Xin Hãy Quên Em', quantity=24, author=[1607], price=59400, description='Xin Hãy Quên Em\nCuộc đời bạn đột ngột thay đổi khi một  mầm sống mới cựa quậy trong lòng bạn. Vui sướng, ngỡ ngàng và cả lo  lắng, thậm chí khủng hoảng, khiếp hãi… là những triệu chứng dễ bắt gặp  từ những bà mẹ đơn thân.\nThật không dễ dàng gì để bắt đầu một  cuộc sống mới! Sau cái chết đột ngột của mẹ, Anna Temple nhận ra rằng cô  cần phải sống và theo đuổi giấc mơ trở thành một nhà thiết kế vườn.  Đánh đổi quỹ phòng ngừa rủi ro cho các bờ cỏ, và cổ phiếu cho cây hoàng  anh ngát hương thơm, cô tạm biệt cuộc sống chốn thành thị xô bồ để bắt  đầu một khởi đầu mới ở vùng nông thôn. \nNhưng trước khi tương lai mới đầy  tươi sáng của cô ùa tới, cô đã gặp Xan – một anh chàng cực kỳ đẹp trai,  và cuộc gặp mặt của họ đã làm thay đổi thế giới của cô theo nhiều cách  mà cô khó có thể tưởng tượng–xuất hiện bé Milly. Việc sắp xếp lại công  việc mới và niềm vui lẫn niềm sợ hãi của việc làm mẹ là  một n

In [36]:
book_repository.add_book(book2)

{'message': 'book created successfully.'}

In [10]:
tmp=book_repository.get_book_by_id(111111)
print(tmp)

id=111111 name='Nhà Giả Kim' quantity=29 author=[] price=71100 description='Nhà Giả Kim Tất cả những trải nghiệm trong chuyến phiêu du theo đuổi vận mệnh của mình đã giúp Santiago thấu hiểu được ý nghĩa sâu xa nhất của hạnh phúc, hòa hợp với vũ trụ và con người.\nTiểu thuyết Nhà giả kim của Paulo Coelho như một câu chuyện cổ tích giản dị, nhân ái, giàu chất thơ, thấm đẫm những minh triết huyền bí của phương Đông. Trong lần xuất bản đầu tiên tại Brazil vào năm 1988, sách chỉ bán được 900 bản. Nhưng, với số phận đặc biệt của cuốn sách dành cho toàn nhân loại, vượt ra ngoài biên giới quốc gia, Nhà giả kim đã làm rung động hàng triệu tâm hồn, trở thành một trong những cuốn sách bán chạy nhất mọi thời đại, và có thể làm thay đổi cuộc đời người đọc.\n“Nhưng nhà luyện kim đan không quan tâm mấy đến những điều ấy. Ông đã từng thấy nhiều người đến rồi đi, trong khi ốc đảo và sa mạc vẫn là ốc đảo và sa mạc. Ông đã thấy vua chúa và kẻ ăn xin đi qua biển cát này, cái biển cát thường xuyên thay hìn

In [48]:
book_repository.update_book(book3)

TypeError: tuple indices must be integers or slices, not str

In [30]:
book_repository.update_book_byID(111111,book2)

{'message': 'book updated successfully.'}

In [40]:
book_repository.delete_book(book2["id"])

{'message': 'book deleted successfully.'}